Graph construction with https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/molecule_net.html
as a template

Simplest GNN few changes to the one from here https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html
as a template

In [ ]:
import csv
import numpy as np
from pysmiles import read_smiles
import networkx as nx
from matplotlib import pyplot as plt
import warnings
from tqdm import tqdm
from torch_geometric.utils.convert import from_networkx
from rdkit import Chem
from pandas import read_csv
from rdkit import Chem
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
warnings.filterwarnings("ignore")

In [ ]:
train_data = read_csv('data_good/train.csv')
test_data = read_csv('data_good/test.csv')

In [ ]:
train_data['Smiles'][0]

In [ ]:
molecules = [Chem.MolFromSmiles(smile) for smile in tqdm(train_data['Smiles'])]
train_data['molecules'] = molecules

molecules = [Chem.MolFromSmiles(smile) for smile in tqdm(test_data['Smiles'])]
test_data['molecules'] = molecules

In [ ]:
def get_single_raw_node_features(atom):
    now = []
    now.append(atom.GetAtomicNum())
    now.append(atom.GetChiralTag())
    now.append(atom.GetTotalDegree())
    now.append(atom.GetFormalCharge())
    now.append(atom.GetTotalNumHs())
    now.append(atom.GetNumRadicalElectrons())
    now.append(atom.GetHybridization())
    now.append(atom.GetIsAromatic())
    now.append(atom.IsInRing())
    return now

def get_raw_node_features(mol):
    features = []
    for atom in mol.GetAtoms():
        features.append(get_single_raw_node_features(atom))
    return features

def get_single_raw_bond_features(bond):
    now = []
    now.append(str(bond.GetBondType()))
    now.append(str(bond.GetStereo()))
    now.append(bond.GetIsConjugated())
    return now

def get_raw_bond_features(mol):
    features = []
    for bond in mol.GetBonds():
        features.append(get_single_raw_bond_features(bond))
    return features

In [ ]:
class Encoder():
    def __init__(self, features, print_unique = True):
        reshaped = [[el] for el in features[0]]
        for i in range(1, len(features)):
            for j in range(len(reshaped)):
                reshaped[j].append(features[i][j])
        self.unique = [list(np.unique(values)) for values in reshaped]
        if (print_unique):
            for i in range(len(self.unique)):
                print(i, self.unique[i])
                
                
    def transform(self, features, show_progress = True):
        result = []
        for vector in tqdm(features, disable = not show_progress):
            now = []
            for i in range(len(vector)):
                current = np.zeros(len(self.unique[i]))
                current[self.unique[i].index(vector[i])] = 1.0
                now.append(current)
            now = np.concatenate(now, axis = 0)
            result.append(now)
        return np.array(result)

In [ ]:
train_raw_bond_features = []
for mol in train_data['molecules']:
    train_raw_bond_features = train_raw_bond_features + get_raw_bond_features(mol)

bond_encoder = Encoder(train_raw_bond_features)

train_bond_features = bond_encoder.transform(train_raw_bond_features)
print(train_bond_features.shape)

In [ ]:
train_raw_node_features = []
for mol in train_data['molecules']:
    train_raw_node_features = train_raw_node_features + get_raw_node_features(mol)
    
node_encoder = Encoder(train_raw_node_features)

train_node_features = node_encoder.transform(train_raw_node_features)
print(train_node_features.shape)




In [ ]:
def mol_2_pytorch_geometric(mol, node_encoder, bond_encoder):
    node_features = node_encoder.transform(get_raw_node_features(mol), show_progress = False)
    #print("node features: ", node_features.shape)
    
    edge_indices, edge_attrs = [], []
    
    num_bond_features = None
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        bond_features = bond_encoder.transform([get_single_raw_bond_features(bond)], show_progress = False)
        num_bond_features = len(bond_features[0])
        #print("bond features: ", bond_features[0].shape)
        edge_indices += [[i, j], [j, i]]
        edge_attrs += [bond_features[0], bond_features[0]]
    
    if len(edge_attrs) > 0:
        
   
        edge_index = torch.tensor(edge_indices)
        edge_index = edge_index.t().to(torch.long).view(2, -1)
        edge_attrs = torch.tensor(edge_attrs, dtype=torch.long).view(-1, num_bond_features)
        ''' print(edge_attrs.shape)
        print(edge_index.shape)
        print(node_features.shape)
        print(edge_index)'''
        # Sort indices.
  
        perm = (edge_index[0] * node_features.shape[0] + edge_index[1]).argsort()
        #print("perm: ", perm.shape)
        #print(edge_index[0] * node_features.shape[0] + edge_index[1])
        edge_index, edge_attrs = edge_index[:, perm], edge_attrs[perm]
        data = Data(x=torch.FloatTensor(node_features), edge_index=edge_index, edge_attr=edge_attrs, empty = False)
    else:
        data = Data(x = torch.FloatTensor(node_features), empty = True)
    #print(edge_index.shape)
    #print(edge_attrs.shape)
    
    return data

In [ ]:
first = mol_2_pytorch_geometric(train_data['molecules'][0], node_encoder, bond_encoder)

In [ ]:
train_data['graphs'] = [mol_2_pytorch_geometric(molecule, node_encoder, bond_encoder) 
                        for molecule in tqdm(train_data['molecules'])]

test_data['graphs'] = [mol_2_pytorch_geometric(molecule, node_encoder, bond_encoder)
                       for molecule in tqdm(test_data['molecules'])]

In [ ]:
positive_indices = np.arange(len(train_data))[np.array(train_data['Active'])]
negative_indices = np.arange(len(train_data))[np.array(train_data['Active']) == False]

np.random.seed(0)
np.random.shuffle(positive_indices)
np.random.shuffle(negative_indices)
print(positive_indices.shape)
print(negative_indices.shape)

In [ ]:
TRAIN_PERCENTAGE = 0.66
num_train_positive = int(positive_indices.shape[0] * TRAIN_PERCENTAGE)
print(num_train_positive)
train_positive = positive_indices[:num_train_positive]
val_positive = positive_indices[num_train_positive:]

num_train_negative = int(negative_indices.shape[0] * TRAIN_PERCENTAGE)
print(num_train_negative)
train_negative = negative_indices[:num_train_negative]
val_negative = negative_indices[num_train_negative:]

In [ ]:
train_indices = np.concatenate([train_positive, train_negative], axis = 0)
val_indices = np.concatenate([val_positive, val_negative], axis = 0)


In [ ]:
val_graphs = [train_data['graphs'].to_list()[index] for index in val_indices]
val_labels = train_data['Active'].to_numpy()[val_indices]

train_graphs = [train_data['graphs'].to_list()[index] for index in train_indices]
train_labels = train_data['Active'].to_numpy()[train_indices]

print(train_labels)
for i in range(len(train_graphs)):
    train_graphs[i].y = int(train_labels[i])
for i in range(len(val_graphs)):
    val_graphs[i].y = int(val_labels[i])

    
train_graphs = [graph for graph in train_graphs if not graph.empty]
val_graphs = [graph for graph in val_graphs if not graph.empty]

In [ ]:
print(len(val_graphs))
print(len(train_graphs))

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.nn import global_max_pool
from torch import nn

class GCN(torch.nn.Module):
    def __init__(self, num_node_features):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.conv2 = GCNConv(128, 128)
        self.conv3 = GCNConv(128, 128)
        self.mlp_first = nn.Sequential(nn.Linear(128, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Linear(128, 128))
        self.mlp = nn.Sequential(nn.Linear(128, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Linear(128, 2))
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p = 0.1, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p = 0.1, training=self.training)
        x = self.conv3(x, edge_index)
        x = self.mlp_first(x)
        x = global_max_pool(x, batch=data.batch)
        x = self.mlp(x)
        x = F.log_softmax(x)
        return x


In [ ]:
BATCH_SIZE = 128
train_loader = DataLoader(train_graphs, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
from sklearn.metrics import f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(54).to(device)
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5, patience = 1000)

NUM_EPOCHS = 100000
for epoch in range(NUM_EPOCHS):
    model.train(True)
    for data in train_loader:
        data.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        #print(outputs)
        #print(data.y)
        loss = F.nll_loss(outputs, data.y)
        #loss = torch.sum(outputs[:, 0])
        loss.backward()
        optimizer.step()


    model.train(False)

    all_outputs = []
    all_targets = []

    for data in val_loader:
        data.to(device)
        outputs = model(data)
        all_outputs.append(outputs.data.cpu().numpy())
        all_targets.append(data.y.cpu().numpy())
    all_outputs = np.concatenate(all_outputs, axis = 0)
    all_targets = np.concatenate(all_targets, axis = 0)

    values = []
    for threshold in np.linspace(np.min(all_outputs), np.max(all_outputs), 1000):
        predictions = all_outputs[:, 1] > threshold
        now = f1_score(all_targets, predictions)
        values.append(now)
    
    lr_scheduler.step(np.max(values))
    
    if epoch % 5 == 0:
        print(epoch, np.max(values), optimizer.param_groups[0]['lr'])
        
   


In [ ]:
model.train(False)

all_outputs = []
all_targets = []

for data in val_loader:
    data.to(device)
    outputs = model(data)
    all_outputs.append(outputs.data.cpu().numpy())
    all_targets.append(data.y.cpu().numpy())
all_outputs = np.concatenate(all_outputs, axis = 0)
all_targets = np.concatenate(all_targets, axis = 0)

values = []
grid = np.linspace(np.min(all_outputs), np.max(all_outputs), 1000)
for threshold in grid:
    predictions = all_outputs[:, 1] > threshold
    now = f1_score(all_targets, predictions)
    values.append(now)
    
plt.plot(grid, values)

In [ ]:
mask = np.logical_and(grid > -20, grid < -1)
values = np.array(values)
plt.plot(grid[mask], values[mask])

In [ ]:
import pandas

In [ ]:
threshold = -8

In [ ]:
test_graphs = test_data['graphs'].to_list()

In [ ]:
test_loader = DataLoader(test_graphs, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
all_outputs = []

for data in test_loader:
    data.to(device)
    outputs = model(data)
    all_outputs.append(outputs.data.cpu().numpy())
all_outputs = np.concatenate(all_outputs, axis = 0)

test_predictions = all_outputs[:, 1] > threshold

In [ ]:
proper = pandas.DataFrame({"Active" : test_predictions})
proper

In [ ]:
proper.to_csv('submission_simplest_gnn.csv')